# Project 2: Data Representations and Clustering

### Group Members
- Shaira Alam

- Vani Agrawal 

- Dhakshina Ilango

# Part 1 - Clustering on Text Data

## Question 1

### 1.1 Report the dimensions of the TF-IDF matrix you obtain.

## Question 2

### 2.1 Report the contingency table of your clustering result. You may use the provided plotmat.py to visualize the matrix.

### 2.2 Does the contingency matrix have to be square-shaped?

## Question 3

### 3.1 Report the 5 clustering measures explained in the introduction for K-means clustering.

## Question 4

### 4.1 Report the plot of the percentage of variance that the top r principle components retain v.s. r, for r = 1 to 1000.

# Part 2 - Deep Learning and Clustering of Image Data

## Question 19

### 19.1 In a brief paragraph discuss: If the VGG network is trained on a dataset with perhaps totally different classes as targets, why would one expect the features derived from such a network to have discriminative power for a custom dataset?